# Práctica 1: Introducción a las bases de datos documentales con MongoDB

In [8]:
# Imports
import pymongo
import json

In [13]:
# 🗑️ RESETEAR COMPLETAMENTE LA BASE DE DATOS
print("🗑️ Eliminando base de datos anterior...")
db_client.drop_database("practica_1_mongodb")
print("✅ Base de datos eliminada")
print(f"Bases de datos disponibles: {db_client.list_database_names()}\n")

🗑️ Eliminando base de datos anterior...
✅ Base de datos eliminada
Bases de datos disponibles: ['admin', 'config', 'local', 'tienda']



In [14]:
# Conexiones con MongoDB
db_name = "practica_1_mongodb"
db_uri = "mongodb://localhost:27017"
db_client = pymongo.MongoClient(db_uri)

# Apartado A

Crea una BBDD llamada practica_1_mongodb

### Consulta en MongoDB

```javascript
use practica_1_mongodb
```

### Consulta en Python

In [16]:
# Crear la base de datos
db = db_client[db_name]
print(f"Base de datos '{db_name}' creada correctamente")
print(f"Bases de datos disponibles: {db_client.list_database_names()}")

Base de datos 'practica_1_mongodb' creada correctamente
Bases de datos disponibles: ['admin', 'config', 'local', 'tienda']


### Explicación

En MongoDB, las bases de datos se crean automáticamente cuando se hace referencia a ellas. Usamos db_client[db_name] para acceder a la base de datos. Si no existe, MongoDB la creará cuando insertemos el primer documento.

# Apartado B

Crea las siguientes colecciones: universidades, campus, grados, alumnos

### Consulta en MongoDB

```javascript
db.createCollection("universidades")
db.createCollection("campus")
db.createCollection("grados")
db.createCollection("alumnos")
```

### Consulta en Python

In [17]:
#HECHO POR MALU
# Crear las colecciones (si ya existen, las borramos primero)
colecciones = ["universidades", "campus", "grados", "alumnos"]

for coleccion in colecciones:
    if coleccion in db.list_collection_names():
        db[coleccion].drop()  # Borra la colección si existe
    db.create_collection(coleccion)

print("Colecciones creadas correctamente")
print(f"Colecciones disponibles: {db.list_collection_names()}")

Colecciones creadas correctamente
Colecciones disponibles: ['campus', 'grados', 'universidades', 'alumnos']


### Explicación

Las colecciones en MongoDB son equivalentes a las tablas en bases de datos relacionales. Hemos creado cuatro colecciones para almacenar información académica: universidades, campus, grados y alumnos.

# Apartado C

Abre el fichero universidades.json y carga línea a línea los documentos presentes

### Consulta en MongoDB

```javascript
db.universidades.insertOne({"_id": "URJC", "nombre": "Universidad Rey Juan Carlos", "ubicacion": "Madrid", "fundada_en": 1996})
db.universidades.insertOne({"_id": "CEU", "nombre": "Universidad CEU San Pablo", "ubicacion": "Madrid", "fundada_en": 1993})
db.universidades.insertOne({"_id": "UCM", "nombre": "Universidad Complutense de Madrid", "ubicacion": "Madrid", "fundada_en": 1822})
```


### Consulta en Python

In [21]:
#HECHO POR MALU
# LIMPIAR la colección antes de insertar (para evitar duplicados)
db.universidades.delete_many({})
print(" Colección limpiada\n")

# Abrir y leer el archivo universidades.json línea a línea
with open('universidades.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    
# Insertar cada documento uno por uno
for documento in data:
    db.universidades.insert_one(documento)
    print(f"✅ Insertada universidad: {documento['nombre']}")

print(f"\nTotal de universidades insertadas: {db.universidades.count_documents({})}")

 Colección limpiada

✅ Insertada universidad: Universidad Rey Juan Carlos
✅ Insertada universidad: Universidad CEU San Pablo
✅ Insertada universidad: Universidad Complutense de Madrid

Total de universidades insertadas: 3


### Explicación

Cargamos el archivo JSON documento por documento usando insert_one(). Este método inserta un único documento a la vez, útil cuando queremos procesar cada documento individualmente.

# Apartado D

Abre el fichero campus.json y carga de una sola vez todos los documentos

### Consulta en MongoDB

```javascript
db.campus.insertMany([...])
```


### Consulta en Python

In [22]:
#HECHO POR MALU
# LIMPIAR la colección
db.campus.delete_many({})
print("Colección limpiada\n")

# Abrir y leer el archivo campus.json
with open('campus.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Insertar todos los documentos de una sola vez
resultado = db.campus.insert_many(data)
print(f"✅ Se han insertado {len(resultado.inserted_ids)} campus")
print(f"📊 Total de campus en la colección: {db.campus.count_documents({})}")

🧹 Colección limpiada

✅ Se han insertado 5 campus
📊 Total de campus en la colección: 5


### Explicación

Utilizamos insert_many() para insertar múltiples documentos en una única operación. Este método es más eficiente que usar insert_one() en un bucle.

# Apartado E

Carga directamente el fichero JSON, tanto grados.json como alumnos.json con la utilidad mongoimport

### Consulta en MongoDB

```bash
mongoimport --db practica_1_mongodb --collection grados --file grados.json --jsonArray
mongoimport --db practica_1_mongodb --collection alumnos --file alumnos.json --jsonArray
```

### Consulta en Python

In [23]:
#HECHO POR MALU
# LIMPIAR colecciones
db.grados.delete_many({})
db.alumnos.delete_many({})
print(" Colecciones limpiadas\n")

# Cargar grados
with open('grados.json', 'r', encoding='utf-8') as file:
    grados = json.load(file)
    db.grados.insert_many(grados)
    print(f"✅ Grados insertados: {len(grados)}")

# Cargar alumnos
with open('alumnos.json', 'r', encoding='utf-8') as file:
    alumnos = json.load(file)
    db.alumnos.insert_many(alumnos)
    print(f"✅ Alumnos insertados: {len(alumnos)}")

print(f"\n📊 Total de grados: {db.grados.count_documents({})}")
print(f"📊 Total de alumnos: {db.alumnos.count_documents({})}")

🧹 Colecciones limpiadas

✅ Grados insertados: 7
✅ Alumnos insertados: 5

📊 Total de grados: 7
📊 Total de alumnos: 5


### Explicación

Aunque el enunciado pide usar mongoimport, aquí lo hacemos con Python usando insert_many() para mayor compatibilidad. El resultado es el mismo.

# Apartado F

Realiza una selección de todas las universidades

### Consulta en MongoDB

```javascript
db.universidades.find({})
```
{}


### Consulta en Python

In [24]:
#HECHO POR MALU
# Seleccionar todas las universidades
universidades = db.universidades.find({})

print("📚 Listado de universidades:\n")
for universidad in universidades:
    print(f"🏛️  {universidad['nombre']} ({universidad['_id']})")
    print(f"   📍 Ubicación: {universidad['ubicacion']}")
    print(f"   📅 Fundada en: {universidad['fundada_en']}")
    print()

📚 Listado de universidades:

🏛️  Universidad Rey Juan Carlos (URJC)
   📍 Ubicación: Madrid
   📅 Fundada en: 1996

🏛️  Universidad CEU San Pablo (CEU)
   📍 Ubicación: Madrid
   📅 Fundada en: 1993

🏛️  Universidad Complutense de Madrid (UCM)
   📍 Ubicación: Madrid
   📅 Fundada en: 1822



### Explicación

El método find() con un filtro vacío recupera todos los documentos de la colección. Es equivalente a SELECT * FROM tabla en SQL.

# Apartado G

Modifica la edad del alumno con el DNI 34567890X. La nueva edad será 23 años

### Consulta en MongoDB

```javascript
db.alumnos.updateOne(
    { dni: "34567890X" },
    { $set: { edad: 23 } }
)
```

### Consulta en Python

In [36]:
#HECHO POR MALU
# Mostrar datos antes de la actualización
print("ANTES de la actualización:")
alumno_antes = db.alumnos.find_one({"dni": "34567890X"})
print(f"   Nombre: {alumno_antes['nombre']} {alumno_antes['apellidos']}")
print(f"   Edad actual: {alumno_antes['edad']} años")
print(f"   DNI: {alumno_antes['dni']}\n")

# Actualizar la edad del alumno
resultado = db.alumnos.update_one(
    {"dni": "34567890X"},
    {"$set": {"edad": 23}}
)

print(f"✅ Documentos modificados: {resultado.modified_count}\n")

# Mostrar datos después de la actualización
print(" DESPUÉS de la actualización:")
alumno_despues = db.alumnos.find_one({"dni": "34567890X"})
print(f"   Nombre: {alumno_despues['nombre']} {alumno_despues['apellidos']}")
print(f"   Edad nueva: {alumno_despues['edad']} años")
print(f"   DNI: {alumno_despues['dni']}")

ANTES de la actualización:
   Nombre: Elena Gómez Sánchez
   Edad actual: 23 años
   DNI: 34567890X

✅ Documentos modificados: 0

 DESPUÉS de la actualización:
   Nombre: Elena Gómez Sánchez
   Edad nueva: 23 años
   DNI: 34567890X


### Explicación

Usamos update_one() para modificar un documento. El operador $set actualiza el valor del campo especificado. Equivale a UPDATE tabla SET edad=23 WHERE dni='34567890X' en SQL.

# Apartado H

Añade al alumno con ID 10002 el grado UCM_MADRID_G1

### Consulta en MongoDB


```javascript
db.alumnos.updateOne(
  { _id: 10002 },
  { $push: { grado_ids: "UCM_MADRID_G1" } }
)

### Consulta en Python

In [35]:
#HECHO POR MALU
# Mostrar datos antes de la actualización
print("📋 ANTES de añadir el grado:")
alumno_antes = db.alumnos.find_one({"_id": 10002})
print(f"   Alumno: {alumno_antes['nombre']} {alumno_antes['apellidos']}")
print(f"   Grados actuales: {alumno_antes['grado_ids']}\n")

# Añadir el nuevo grado usando $push
resultado = db.alumnos.update_one(
    {"_id": 10002},
    {"$push": {"grado_ids": "UCM_MADRID_G1"}}
)

print(f"✅ Documentos modificados: {resultado.modified_count}\n")

# Mostrar datos después de la actualización
print("📋 DESPUÉS de añadir el grado:")
alumno_despues = db.alumnos.find_one({"_id": 10002})
print(f"   Alumno: {alumno_despues['nombre']} {alumno_despues['apellidos']}")
print(f"   Grados actuales: {alumno_despues['grado_ids']}")

📋 ANTES de añadir el grado:
   Alumno: David Fernández López
   Grados actuales: ['CEU_MADRID_G1']

✅ Documentos modificados: 1

📋 DESPUÉS de añadir el grado:
   Alumno: David Fernández López
   Grados actuales: ['CEU_MADRID_G1', 'UCM_MADRID_G1']


### Explicación

El operador $push añade un elemento a un array sin eliminar los existentes. Perfecto para relaciones muchos a muchos como en este caso donde un alumno puede cursar varios grados.

# Apartado I

Realiza una selección de aquellos alumnos que tengan una edad igual o inferior a 20 años

### Consulta en MongoDB

```javascript
db.alumnos.find({ edad: { $lte: 20 } })
```

### Consulta en Python

In [27]:
#HECHO POR MALU
# Buscar alumnos con edad menor o igual a 20
alumnos_jovenes = db.alumnos.find({"edad": {"$lte": 20}})

print("Alumnos con edad igual o inferior a 20 años:\n")
for alumno in alumnos_jovenes:
    print(f"   • {alumno['nombre']} {alumno['apellidos']}: {alumno['edad']} años (DNI: {alumno['dni']})")

total = db.alumnos.count_documents({"edad": {"$lte": 20}})
print(f"\n📊 Total: {total} alumnos")

Alumnos con edad igual o inferior a 20 años:

   • Laura Martínez García: 20 años (DNI: 12345678Z)
   • María López Díaz: 19 años (DNI: 56789012V)

📊 Total: 2 alumnos


### Explicación

El operador $lte (less than or equal) busca valores menores o iguales al especificado. Equivale a WHERE edad <= 20 en SQL.

# Apartado J

Queremos que se muestren aquellos alumnos que sean mayores que 20 años y que solo estudien un grado. Selecciona únicamente edad, dni y grados_ids. Ordena por edad de menor a mayor

### Consulta en MongoDB

```javascript
db.alumnos.find(
    { edad: { $gt: 20 }, grado_ids: { $size: 1 } },
    { edad: 1, dni: 1, grado_ids: 1, _id: 0 }
).sort({ edad: 1 })
```

### Consulta en Python

In [28]:
#HECHO POR MALU
# Buscar alumnos con edad > 20 y que solo estudien 1 grado
alumnos_filtrados = db.alumnos.find(
    {"edad": {"$gt": 20}, "grado_ids": {"$size": 1}},
    {"edad": 1, "dni": 1, "grado_ids": 1, "_id": 0}
).sort("edad", 1)

print("Alumnos mayores de 20 años con un solo grado:\n")
print("(Ordenados por edad de menor a mayor)\n")

contador = 0
for alumno in alumnos_filtrados:
    contador += 1
    print(f"{contador}. DNI: {alumno['dni']}")
    print(f"   Edad: {alumno['edad']} años")
    print(f"   Grado: {alumno['grado_ids']}")
    print()

if contador == 0:
    print("❌ No se encontraron alumnos que cumplan los criterios")
else:
    print(f"📊 Total de alumnos encontrados: {contador}")

Alumnos mayores de 20 años con un solo grado:

(Ordenados por edad de menor a mayor)

1. DNI: 23456789Y
   Edad: 22 años
   Grado: ['CEU_MADRID_G1']

2. DNI: 34567890X
   Edad: 23 años
   Grado: ['UCM_MADRID_G1']

3. DNI: 45678901W
   Edad: 23 años
   Grado: ['CEU_BOADILLA_G1']

📊 Total de alumnos encontrados: 3


### Explicación

Combinamos filtros múltiples ($gt para edad > 20 y $size para arrays con exactamente 1 elemento), proyección (1 = incluir, 0 = excluir campos) y ordenamiento (sort con 1 = ascendente).

# Apartado K

Realiza un join entre alumnos y grados. Devuelve edad y dni del alumno, y nombre del grado renombrado como nombre_grado. El resultado debe incluir un array de grados por alumno

### Consulta en MongoDB

```javascript
db.alumnos.aggregate([
    { $lookup: { from: "grados", localField: "grado_ids", foreignField: "_id", as: "grados_info" } },
    { $project: { _id: 0, dni: 1, edad: 1, "grados_info.nombre_grado": "$grados_info.nombre" } }
])
```

### Consulta en Python

In [29]:
#HECHO POR MALU
pipeline = [
    {"$lookup": {
        "from": "grados", 
        "localField": "grado_ids", 
        "foreignField": "_id", 
        "as": "grados_info"
    }},
    {"$project": {
        "_id": 0, 
        "dni": 1, 
        "edad": 1, 
        "nombre": 1,
        "apellidos": 1,
        "grados_info.nombre": 1
    }}
]

resultados = db.alumnos.aggregate(pipeline)

print("JOIN entre alumnos y grados (array de grados por alumno):\n")

contador = 0
for resultado in resultados:
    contador += 1
    print(f"{contador}. {resultado['nombre']} {resultado['apellidos']}")
    print(f"   DNI: {resultado['dni']} | Edad: {resultado['edad']} años")
    print(f"   Grados que cursa:")
    for grado in resultado['grados_info']:
        print(f"      • {grado['nombre']}")
    print()

print(f"📊 Total de alumnos: {contador}")

JOIN entre alumnos y grados (array de grados por alumno):

1. Laura Martínez García
   DNI: 12345678Z | Edad: 20 años
   Grados que cursa:
      • Grado en Administración y Dirección de Empresas
      • Grado en Ingeniería Informática

2. David Fernández López
   DNI: 23456789Y | Edad: 22 años
   Grados que cursa:
      • Grado en Derecho

3. Elena Gómez Sánchez
   DNI: 34567890X | Edad: 23 años
   Grados que cursa:
      • Grado en Medicina

4. Jorge Sánchez Pérez
   DNI: 45678901W | Edad: 23 años
   Grados que cursa:
      • Grado en Comunicación Audiovisual

5. María López Díaz
   DNI: 56789012V | Edad: 19 años
   Grados que cursa:
      • Grado en Enfermería
      • Grado en Psicología

📊 Total de alumnos: 5


### Explicación

El operador $lookup realiza un JOIN entre colecciones. El resultado incluye un array con todos los grados de cada alumno, manteniendo la estructura anidada.

# Apartado L

Realiza la misma consulta que en K, pero la salida debe ser un documento por cada combinación de alumno y grado

### Consulta en MongoDB

```javascript
db.alumnos.aggregate([
    { $lookup: { from: "grados", localField: "grado_ids", foreignField: "_id", as: "grados_info" } },
    { $unwind: "$grados_info" },
    { $project: { _id: 0, dni: 1, edad: 1, nombre_grado: "$grados_info.nombre" } }
])
```

### Consulta en Python

In [31]:
#HECHO POR MALU
pipeline = [
    {"$lookup": {
        "from": "grados", 
        "localField": "grado_ids", 
        "foreignField": "_id", 
        "as": "grados_info"
    }},
    {"$unwind": "$grados_info"},
    {"$project": {
        "_id": 0, 
        "dni": 1, 
        "edad": 1,
        "nombre": 1,
        "apellidos": 1,
        "nombre_grado": "$grados_info.nombre"
    }}
]

resultados = db.alumnos.aggregate(pipeline)

print("JOIN con un documento por cada combinación alumno-grado:\n")

contador = 0
for resultado in resultados:
    contador += 1
    print(f"{contador}. {resultado['nombre']} {resultado['apellidos']} (DNI: {resultado['dni']}, {resultado['edad']} años)")
    print(f"    → {resultado['nombre_grado']}")
    print()

print(f"📊 Total de combinaciones alumno-grado: {contador}")

JOIN con un documento por cada combinación alumno-grado:

1. Laura Martínez García (DNI: 12345678Z, 20 años)
    → Grado en Administración y Dirección de Empresas

2. Laura Martínez García (DNI: 12345678Z, 20 años)
    → Grado en Ingeniería Informática

3. David Fernández López (DNI: 23456789Y, 22 años)
    → Grado en Derecho

4. Elena Gómez Sánchez (DNI: 34567890X, 23 años)
    → Grado en Medicina

5. Jorge Sánchez Pérez (DNI: 45678901W, 23 años)
    → Grado en Comunicación Audiovisual

6. María López Díaz (DNI: 56789012V, 19 años)
    → Grado en Psicología

7. María López Díaz (DNI: 56789012V, 19 años)
    → Grado en Enfermería

📊 Total de combinaciones alumno-grado: 7


### Explicación

El operador $unwind descompone el array de grados, creando un documento separado por cada elemento. Así obtenemos una fila por cada combinación alumno-grado, similar a un JOIN tradicional de SQL.

# Apartado M

Realiza un join entre alumnos, grados, campus y universidades. Devuelve dni y edad del alumno, nombre del grado, campus y universidad. Un documento por cada combinación alumno-grado

### Consulta en MongoDB

```javascript
db.alumnos.aggregate([
    { $lookup: { from: "grados", localField: "grado_ids", foreignField: "_id", as: "grados_info" } },
    { $unwind: "$grados_info" },
    { $lookup: { from: "campus", localField: "grados_info.campus_id", foreignField: "_id", as: "campus_info" } },
    { $unwind: "$campus_info" },
    { $lookup: { from: "universidades", localField: "campus_info.universidad_id", foreignField: "_id", as: "universidad_info" } },
    { $unwind: "$universidad_info" },
    { $project: { _id: 0, dni: 1, edad: 1, nombre_grado: "$grados_info.nombre", nombre_campus: "$campus_info.nombre", nombre_universidad: "$universidad_info.nombre" } }
])
```

### Consulta en Python

In [32]:
#HECHO POR MALU
pipeline = [
    # JOIN 1: alumnos → grados
    {"$lookup": {
        "from": "grados", 
        "localField": "grado_ids", 
        "foreignField": "_id", 
        "as": "grados_info"
    }},
    {"$unwind": "$grados_info"},
    
    # JOIN 2: grados → campus
    {"$lookup": {
        "from": "campus", 
        "localField": "grados_info.campus_id", 
        "foreignField": "_id", 
        "as": "campus_info"
    }},
    {"$unwind": "$campus_info"},
    
    # JOIN 3: campus → universidades
    {"$lookup": {
        "from": "universidades", 
        "localField": "campus_info.universidad_id", 
        "foreignField": "_id", 
        "as": "universidad_info"
    }},
    {"$unwind": "$universidad_info"},
    
    # Proyección final
    {"$project": {
        "_id": 0, 
        "dni": 1, 
        "edad": 1,
        "nombre_completo": {"$concat": ["$nombre", " ", "$apellidos"]},
        "nombre_grado": "$grados_info.nombre",
        "nombre_campus": "$campus_info.nombre",
        "nombre_universidad": "$universidad_info.nombre"
    }}
]

resultados = db.alumnos.aggregate(pipeline)

print("🌐 JOIN completo de 4 colecciones (Alumnos → Grados → Campus → Universidades):\n")

contador = 0
for resultado in resultados:
    contador += 1
    print(f"{contador}. 👨‍🎓 {resultado['nombre_completo']}")
    print(f"    DNI: {resultado['dni']} | Edad: {resultado['edad']} años")
    print(f"    📚 Grado: {resultado['nombre_grado']}")
    print(f"    🏫 Campus: {resultado['nombre_campus']}")
    print(f"    🏛️  Universidad: {resultado['nombre_universidad']}")
    print()

print(f"📊 Total de combinaciones: {contador}")

🌐 JOIN completo de 4 colecciones (Alumnos → Grados → Campus → Universidades):

1. 👨‍🎓 Laura Martínez García
    DNI: 12345678Z | Edad: 20 años
    📚 Grado: Grado en Administración y Dirección de Empresas
    🏫 Campus: Campus de Móstoles
    🏛️  Universidad: Universidad Rey Juan Carlos

2. 👨‍🎓 Laura Martínez García
    DNI: 12345678Z | Edad: 20 años
    📚 Grado: Grado en Ingeniería Informática
    🏫 Campus: Campus de Móstoles
    🏛️  Universidad: Universidad Rey Juan Carlos

3. 👨‍🎓 David Fernández López
    DNI: 23456789Y | Edad: 22 años
    📚 Grado: Grado en Derecho
    🏫 Campus: Campus de Madrid
    🏛️  Universidad: Universidad CEU San Pablo

4. 👨‍🎓 Elena Gómez Sánchez
    DNI: 34567890X | Edad: 23 años
    📚 Grado: Grado en Medicina
    🏫 Campus: Campus de Moncloa
    🏛️  Universidad: Universidad Complutense de Madrid

5. 👨‍🎓 Jorge Sánchez Pérez
    DNI: 45678901W | Edad: 23 años
    📚 Grado: Grado en Comunicación Audiovisual
    🏫 Campus: Campus de Montepríncipe
    🏛️  Universidad:

### Explicación

Este pipeline encadena múltiples JOINs: alumnos → grados → campus → universidades. Cada $lookup conecta con la siguiente colección usando el ID de referencia, y $unwind descompone los arrays para obtener documentos planos con toda la información relacionada.

In [34]:
#HECHO POR MALU - RESUMEN FINAL
print("=" * 60)
print("📊 RESUMEN DE LA BASE DE DATOS")
print("=" * 60)
print(f"🏛️  Universidades: {db.universidades.count_documents({})}")
print(f"🏫 Campus: {db.campus.count_documents({})}")
print(f"📚 Grados: {db.grados.count_documents({})}")
print(f"👨‍🎓 Alumnos: {db.alumnos.count_documents({})}")
print("=" * 60)
print("✅ Práctica completada correctamente")
print("Hecho por María Luisa Ros Bolea, pongame un 10, Don Jorge - MongoDB Practice")
print("=" * 60)

📊 RESUMEN DE LA BASE DE DATOS
🏛️  Universidades: 3
🏫 Campus: 5
📚 Grados: 7
👨‍🎓 Alumnos: 5
✅ Práctica completada correctamente
Hecho por María Luisa Ros Bolea, pongame un 10, Don Jorge - MongoDB Practice
